In [33]:
import json
from pathlib2 import Path
from pprint import pprint

ingredient_file = 'ewg_ingredients.json'
product_file = 'ewg_products.json'

c1 = json.load(open('ewg_skindeep_ingredients1.json'))
c2 = json.load(open('ewg_skindeep_ingredients2.json'))
c3 = json.load(open('ewg_skindeep_ingredients3.json'))

products = {}
ingredients = {}
dup_prods = None
dup_ing = None

In [34]:
# Collect crawled data
# Since 3 crawler instances were ran together asynchronously
# concatenate results and remove duplicates

# Only parse ingredients and product if this step hasn't already been done 
if not Path(product_file).is_file():
    dup_prods = 0
    p_list = (
        [x for x in c1 if 'product_id' in x.keys()]
        + [x for x in c2 if 'product_id' in x.keys()]
        + [x for x in c3 if 'product_id' in x.keys()])
    for product in p_list:
        if product['product_id'] not in products.keys():
            products[product['product_id']] = product
        else:
            dup_prods = dup_prods + 1
    with open('ewg_products.json', 'w') as f:
        json.dump(products, f)
else:
    products = json.load(open(product_file))

if not Path(ingredient_file).is_file():
    dup_ing = 0
    i_list = (
        [x for x in c1 if 'ingredient_id' in x.keys()]
        + [x for x in c3 if 'ingredient_id' in x.keys()]
        + [x for x in c3 if 'ingredient_id' in x.keys()])
    for ingredient in i_list:
        if ingredient['ingredient_id'] not in ingredients.keys():
            ingredients[ingredient['ingredient_id']] = ingredient
        else:
            dup_ing = dup_ing + 1
    with open('ewg_ingredients.json', 'w') as f:
        json.dump(ingredients, f)
else:
    ingredients = json.load(open(ingredient_file))

print("Crawled data contains {} ingredients in {} unique products".format(len(ingredients), len(products)))
if dup_ing is not None:
    print("Duplicate Ingredients: {}".format(dup_ing))
if dup_prods is not None:
    print("Duplicate Products: {}".format(dup_prods))

Crawled data contains 7349 ingredients in 72247 unique products
Duplicate Products: 1560


In [63]:
count = 0
num_ingredients = 0
for (k, v) in products.iteritems():
    if "ingredient_list" in v.keys():
        count = count + 1
        num_ingredients = num_ingredients + len(v["ingredient_list"])
    #else:
    #    pprint(v)
print("{} out of {} product entries have ingredient lists. The average number of ingredients is {}".format(count, len(products), 1.0 * num_ingredients/count))

72242 out of 72247 product entries have ingredient lists. The average number of ingredients is 21.118310678


In [84]:
# Try cross comparing a product ingredient list to ingredients dictionary
error_keys = []
error_prod = []
for (k, v) in products.iteritems():
    if 'ingredient_list' in v.keys():
        for key in v['ingredient_list']:
            if key not in error_keys:                
                error_keys.append(key)
                error_prod.append(v['url'])

In [89]:
print("Number of keys not in ingredients dict: {}".format(len(error_keys)))
# exmples:
print(error_prod[0])
print(error_prod[300])
print(error_prod[8924])

Number of keys not in ingredients dict: 8925
http://www.ewg.org/skindeep/product/497598/Ella%27s_Botanicals_Chai_Latte_Beauty_Bar_Soap_%28old_formulation%29/
http://www.ewg.org/skindeep/product/535568/Ahava_Mineral_Body_Lotion_%28old_formulation%29/
http://www.ewg.org/skindeep/product/468332/Arcona_Night_Breeze%2C_Heal%3B%3BProtect_PM/
